<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }
</style>

<div class="note-box">
    <h2>👁️ Chapter 4: Object Detection with YOLO</h2>
    <p>
        Welcome to <strong>Notebook 04</strong>. In the previous chapter, we did <strong>Image Classification</strong> (assigning one label to a whole image, e.g., "This is a cat").
    </p>
    <p>
        Now we move to <strong>Object Detection</strong>. This is much harder because the model must answer two questions:
    </p>
    <ol>
        <li><strong>What</strong> is in the image? (Classification)</li>
        <li><strong>Where</strong> is it? (Localization / Bounding Box)</li>
    </ol>
    <p>
        We will use <strong>YOLOv8 (You Only Look Once)</strong>, a state-of-the-art model famous for its incredible speed and accuracy.
    </p>
</div>

<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }
</style>

<div class="note-box">
    <h2>⚙️ Part 1: Setup & Hardware</h2>
    <p>
        We use the <span class="code-inline">ultralytics</span> library, which makes using YOLO models extremely easy.
    </p>
    <p>
        <strong>Apple Silicon Note:</strong> YOLOv8 supports the <strong>MPS</strong> backend natively. We will confirm your GPU is active below.
    </p>
</div>

In [2]:
import torch
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# 1. Check Device
if torch.backends.mps.is_available():
    device = "mps"
    print("✅ Using Apple GPU (MPS backend)")
else:
    device = "cpu"
    print("⚠️ Using CPU")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/Users/heliamirhosseini/Library/Application Support/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Using Apple GPU (MPS backend)


<style>
    /* Main container style */
    .note-box {
        background-color: #1e1e2e;       /* Dark Blue-Grey Background */
        color: #cdd6f4;                  /* Soft White Text */
        border-left: 6px solid #89b4fa;  /* Blue Accent Border */
        border-radius: 8px;
        padding: 20px;
        margin: 20px 0;
        font-family: system-ui, -apple-system, sans-serif;
        line-height: 1.6;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);
        box-sizing: border-box;
        max-width: 100%;
        overflow-wrap: break-word;
    }
    
    /* Header style */
    .note-box h2 {
        color: #89b4fa;                  /* Blue Header */
        margin-top: 0;
        margin-bottom: 15px;
        font-size: 1.6rem;
        font-weight: 600;
        border-bottom: 1px solid #45475a;
        padding-bottom: 10px;
    }

    .note-box h3 {
        color: #f9e2af;                  /* Yellow Sub-Header */
        margin-top: 20px;
        margin-bottom: 10px;
        font-size: 1.2rem;
        font-weight: 600;
    }

    /* Important keywords */
    .note-box strong {
        color: #f9e2af;                  /* Soft Gold/Yellow */
        font-weight: 600;
    }

    /* Inline code snippets */
    .note-box .code-inline {
        background-color: #313244;
        color: #f38ba8;                  /* Soft Red/Pink */
        padding: 2px 6px;
        border-radius: 4px;
        font-family: 'Menlo', 'Consolas', monospace;
        font-size: 0.9em;
        border: 1px solid #45475a;
        white-space: pre-wrap;
    }

    /* Lists */
    .note-box ul {
        padding-left: 20px;
        margin: 10px 0;
    }
    .note-box li {
        margin-bottom: 8px;
    }

    /* Images */
    .note-box img {
        display: block;
        margin: 15px auto;
        max-width: 80%;
        border-radius: 8px;
        border: 1px solid #45475a;
    }
</style>

<div class="note-box">
    <h2>🧠 Theory: How Detection Works</h2>
    <p>
        Before we run the code, we need to understand two core concepts that make Object Detection possible.
    </p>
    
<h3>1. Anchor Boxes</h3>
    <p>
        In the early days, computers scanned every single pixel to find objects, which was incredibly slow.
    </p>
    <p>
        <strong>YOLO</strong> uses a smarter approach. It divides the image into a grid (e.g., 7x7). In each grid cell, it places pre-defined rectangles called <strong>Anchor Boxes</strong> of different shapes (tall and thin for a person, wide for a car).
    </p>
    
<p>
        The network doesn't predict a box from scratch; it predicts <em>how much to stretch or shift</em> these existing Anchor Boxes to fit the object it sees.
    </p>

<h3>2. IoU (Intersection over Union)</h3>
<p>
        How do we know if a prediction is "good"? We use <strong>IoU</strong>.
    </p>
    
<ul>
        <li><strong>Intersection:</strong> The area where the Predicted Box and the Truth Box overlap.</li>
        <li><strong>Union:</strong> The total area covered by both boxes combined.</li>
        <li><strong>IoU Score:</strong> Intersection ÷ Union.</li>
    </ul>
<p>
        An IoU of <strong>1.0</strong> is a perfect match. An IoU > 0.5 is usually considered a "correct" detection.
    </p>
</div>